In [65]:
import pandas as pd
a = [{"name":"a", "age":1}, {"name":"b", "age":2}]
df = pd.DataFrame(a)
df

b = {"name":["가1","나1","다1"], "age":[1,2,3]}
df1 = pd.DataFrame(b)
df1

c = [["가1", 34],["나1",44],["다1", 54]]
df2 = pd.DataFrame(c, columns=["name","age"])
df2

,name,age
0,가1,34
1,나1,44
2,다1,54


In [121]:
import urllib.request as RQ
import xml.etree.ElementTree as ET
import pandas as pd

url = "http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=108"

# 웹에서 데이터를 받음
response = RQ.urlopen(url).read()

# 타입 변경 (bytes -> xml.etree.ElementTree.Element)
root = ET.fromstring(response)

# [{   },{    },{    }]
arr = []
for idx, tmp in enumerate(root.findall('channel/item/description/body/location')):
    for tmp1 in tmp.findall('data'):
        dic = { 'city'    : tmp.findtext('city'),
                'tmEf'    : tmp1.findtext('tmEf'),
                'wf'      : tmp1.findtext('wf'),
                'tmn'     : tmp1.findtext('tmn'),
                'tmx'     : tmp1.findtext('tmx'),
                'rnSt'    : tmp1.findtext('rnSt') }
        arr.append(dic)
        
# print(arr)
df = pd.DataFrame(arr)

In [154]:
import requests
import bs4
import pandas as pd

url = "http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=108"
response = requests.get(url)

soup = bs4.BeautifulSoup(response.text, 'lxml-xml')
soup

arr = []
for idx, tmp in enumerate(soup.select("location")):
    for idx1, tmp1 in enumerate(tmp.select("data")):
        arr.append( [tmp.select_one('city').text,
                  tmp1.select_one("tmEf").text, 
                  tmp1.select_one("wf").text,
                  tmp1.select_one("tmn").text,
                  tmp1.select_one("tmx").text,
                  tmp1.select_one("rnSt").text] )
        
df = pd.DataFrame(arr, columns=["city","tmef","wf","tmn","tmx","rnSt"])
df

# c = []
# for idx, tmp in enumerate(soup.select("location")):
#     for idx1, tmp1 in enumerate(tmp.select("data")):
#         c.append( [tmp.select_one('city').text,
#                   tmp1.select_one("tmef").text, 
#                   tmp1.select_one("wf").text,
#                   tmp1.select_one("tmn").text,
#                   tmp1.select_one("tmx").text,
#                   tmp1.select_one("rnSt").text] )

# df4 = pd.DataFrame(c, columns=["city","tmef","wf","tmn","tmx","rnSt"])
# df4

,city,tmef,wf,tmn,tmx,rnSt
0,서울,2021-05-30 00:00,맑음,15,25,20
1,서울,2021-05-30 12:00,맑음,15,25,10
2,서울,2021-05-31 00:00,맑음,17,27,20
3,서울,2021-05-31 12:00,맑음,17,27,10
4,서울,2021-06-01 00:00,구름많음,18,26,30
...,...,...,...,...,...,...
528,서귀포,2021-06-03 00:00,흐림,20,24,40
529,서귀포,2021-06-03 12:00,흐림,20,24,40
530,서귀포,2021-06-04 00:00,구름많음,19,24,30
531,서귀포,2021-06-05 00:00,맑음,19,25,0


In [155]:
df.columns

Index(['city', 'tmef', 'wf', 'tmn', 'tmx', 'rnSt'], dtype='object')

In [156]:
df.index

RangeIndex(start=0, stop=533, step=1)

In [157]:
df.dtypes

city    object
tmef    object
wf      object
tmn     object
tmx     object
rnSt    object
dtype: object

In [160]:
df = df.astype({"tmef":"datetime64", "tmn":"int64", "tmx":"int64", "rnSt":"int64"})

In [161]:
df.describe()

,tmn,tmx,rnSt
count,533.000000,533.000000,533.000000
mean,15.906191,25.500938,19.493433
std,1.549502,1.700978,12.074582
min,12.000000,20.000000,0.000000
25%,15.000000,24.000000,10.000000
50%,16.000000,26.000000,20.000000
75%,17.000000,27.000000,30.000000
max,20.000000,29.000000,40.000000


In [162]:
df.head()

,city,tmef,wf,tmn,tmx,rnSt
0,서울,2021-05-30 00:00:00,맑음,15,25,20
1,서울,2021-05-30 12:00:00,맑음,15,25,10
2,서울,2021-05-31 00:00:00,맑음,17,27,20
3,서울,2021-05-31 12:00:00,맑음,17,27,10
4,서울,2021-06-01 00:00:00,구름많음,18,26,30


In [163]:
df[['city','wf']]

,city,wf
0,서울,맑음
1,서울,맑음
2,서울,맑음
3,서울,맑음
4,서울,구름많음
...,...,...
528,서귀포,흐림
529,서귀포,흐림
530,서귀포,구름많음
531,서귀포,맑음


In [164]:
df[['city']]

,city
0,서울
1,서울
2,서울
3,서울
4,서울
...,...
528,서귀포
529,서귀포
530,서귀포
531,서귀포


In [165]:
# 데이터 수집 -> 데이터프레임(pandas) -> 데이터탐색(시각화) -> 전처리(데이터의 가공) -> 스케일러(숫자들의 가중치를 일정하게 부여) 
# -> 실수형의 숫자로(numpy) -> 훈련데이터(80%), 테스트데이터 분할(20%) -> 머신러닝 -> 학습된 결과 가중치 -> 테스트데이터를 통해 검증 -> 학습모델 개선(과적합, 과소적합)
# -> 이미지, 음성 -> 딥러닝 -> 인공지능
cont = (df['city'] == '서울')
cont = (df['wf'] == '맑음')
cont = (df['tmn'] == 15) & (df['wf'] == '맑음')
cont = (df['tmn'] == 15) | (df['tmx'] == '25')
df.loc[cont]
# df.loc[cont,['city']]

,city,tmef,wf,tmn,tmx,rnSt
0,서울,2021-05-30 00:00:00,맑음,15,25,20
1,서울,2021-05-30 12:00:00,맑음,15,25,10
13,인천,2021-05-30 00:00:00,맑음,15,23,20
14,인천,2021-05-30 12:00:00,맑음,15,23,10
28,수원,2021-05-31 00:00:00,맑음,15,27,20
...,...,...,...,...,...,...
493,울진,2021-06-06 00:00:00,맑음,15,22,10
494,울릉도,2021-05-30 00:00:00,맑음,15,21,10
495,울릉도,2021-05-30 12:00:00,맑음,15,21,0
496,울릉도,2021-05-31 00:00:00,맑음,15,21,10


In [166]:
df

,city,tmef,wf,tmn,tmx,rnSt
0,서울,2021-05-30 00:00:00,맑음,15,25,20
1,서울,2021-05-30 12:00:00,맑음,15,25,10
2,서울,2021-05-31 00:00:00,맑음,17,27,20
3,서울,2021-05-31 12:00:00,맑음,17,27,10
4,서울,2021-06-01 00:00:00,구름많음,18,26,30
...,...,...,...,...,...,...
528,서귀포,2021-06-03 00:00:00,흐림,20,24,40
529,서귀포,2021-06-03 12:00:00,흐림,20,24,40
530,서귀포,2021-06-04 00:00:00,구름많음,19,24,30
531,서귀포,2021-06-05 00:00:00,맑음,19,25,0


In [167]:
df['tmn_tmx'] = (df['tmn'] + df['tmx'])/2
df

,city,tmef,wf,tmn,tmx,rnSt,tmn_tmx
0,서울,2021-05-30 00:00:00,맑음,15,25,20,20.0
1,서울,2021-05-30 12:00:00,맑음,15,25,10,20.0
2,서울,2021-05-31 00:00:00,맑음,17,27,20,22.0
3,서울,2021-05-31 12:00:00,맑음,17,27,10,22.0
4,서울,2021-06-01 00:00:00,구름많음,18,26,30,22.0
...,...,...,...,...,...,...,...
528,서귀포,2021-06-03 00:00:00,흐림,20,24,40,22.0
529,서귀포,2021-06-03 12:00:00,흐림,20,24,40,22.0
530,서귀포,2021-06-04 00:00:00,구름많음,19,24,30,21.5
531,서귀포,2021-06-05 00:00:00,맑음,19,25,0,22.0


In [168]:
df.groupby('city').mean() #도시별 평균값

,tmn,tmx,rnSt,tmn_tmx
city,,,,
강릉,16.384615,24.000000,23.846154,20.192308
거창,13.615385,26.076923,15.384615,19.846154
경주,14.384615,26.307692,16.153846,20.346154
고창,15.923077,25.538462,19.230769,20.730769
광양,16.769231,26.000000,18.461538,21.384615
광주,16.923077,27.384615,18.461538,22.153846
군산,16.000000,25.384615,19.230769,20.692308
나주,15.307692,27.153846,18.461538,21.230769
남원,15.461538,26.769231,19.230769,21.115385
